### 第13课：动手制作自己的简易聊天机器人

自动问答主要研究的内容和关键科学问题如下：

问句理解：给定用户问题，自动问答首先需要理解用户所提问题。用户问句的语义理解包含词法分析、句法分析、语义分析等多项关键技术，需要从文本的多个维度理解其中包含的语义内容。

文本信息抽取：自动问答系统需要在已有语料库、知识库或问答库中匹配相关的信息，并抽取出相应的答案。

知识推理：自动问答中，由于语料库、知识库和问答库本身的覆盖度有限，并不是所有问题都能直接找到答案。这就需要在已有的知识体系中，通过知识推理的手段获取这些隐含的答案。

In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
Chinese_bot = ChatBot("Training demo") #创建一个新的实例
Chinese_bot.set_trainer(ListTrainer)
Chinese_bot.train([
    '亲，在吗？',
    '亲，在呢',
    '这件衣服的号码大小标准吗？',
    '亲，标准呢，请放心下单吧。',
    '有红色的吗？',
    '有呢，目前有白红蓝3种色调。',
])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zhangjianfeng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhangjianfeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhangjianfeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: 'ChatBot' object has no attribute 'set_trainer'

In [ ]:
# 测试一下
question = '亲，在吗'
print(question)
response = Chinese_bot.get_response(question)
print(response)
print("\n")
question = '有红色的吗？'
print(question)
response = Chinese_bot.get_response(question)
print(response)

In [1]:
lines = open("../data/13/QQ.txt","r",encoding='gbk').readlines()
sec = [ line.strip() for line in lines]

In [2]:
sec

['那你一定没看过编译原理',
 '上过最恶心的专业课',
 '没有之一',
 '觉得对现在工作没用',
 '不会一点影响都没',
 '学习不是为了有用而学',
 '有谁搞过网络拓扑自动发现啊',
 '学习是因为学习使我快乐',
 '不然我早就跑了',
 '真心羡慕那些自学能力强的人',
 '这是那个做了宾果消消乐的那个单位的校招宣传海报',
 '但是',
 '在哪里啊',
 '叫什么柠檬啥啥公司、记不清了',
 '北京',
 '是基本工资么',
 '这个薪资',
 '税后',
 '我总觉得人家大专出去的',
 '北京不算高',
 '也有这么多',
 '我05年的时候去北京就是9000多',
 '税后',
 '6',
 '毕竟大佬儿',
 '[表情]还是基本工资',
 '05年的9000+ 和现在的1w',
 '那简直不是一个概念',
 '现在研究生都不值钱',
 '努努力在北京买房啊',
 '那我这个本科生能出去干什么..',
 '双非..还是 第三批次本科',
 '挂机等死',
 '大专',
 '那死定了啊',
 '6月份满2年',
 '兄弟',
 '挂机、不如浪一波',
 '小学呢',
 '现在税后8k',
 '小学毕业进的培训班',
 '现在10多年经验了',
 '啥都会',
 '初中就去搞代码？怕不是猝死哦',
 '就有人吹牛逼，小学开始写代码',
 '写汇编',
 '可牛逼了',
 '高中就赚外快了',
 '大学就创业了，当cto',
 '大一时我特么还只会hello world',
 '大二呢、',
 '大二的时候我去参加了一个算法比赛',
 '个人见解啊。现在不管做java还是大数据，项目需求一定要和智能，决策这种概念搭点边。',
 '项目价值才大。',
 '如果还是一味做老的数据操作，增删改查，是不值钱。',
 '拿了省二奖',
 '良_datamining_深圳 会个for循环了啥的',
 '大三看hadoop文档',
 '发现都是天书',
 '什么都不会',
 '只会吹水',
 '大学基本上都在学数学证明',
 '可爽了',
 '数学专业？',
 '嗯',
 '那挺好的',
 '我现在想回学校学数学',
 '本科中工作你试试',
 '计算机有数学底子太棒了',
 '看着各种算法好累',
 '数学转计算机转挺好转的',
 '有时候根本看不懂',


In [3]:
# from chatterbot import ChatBot
# from chatterbot.trainers import ListTrainer
# Chinese_bot = ChatBot("Training")
# Chinese_bot.set_trainer(ListTrainer)
# Chinese_bot.train(sec)

Seq2Seq 属于 Encoder-Decoder 结构，这里看看常见的 Encoder-Decoder 结构。基本思想就是利用两个 RNN，一个 RNN 作为 Encoder，另一个 RNN 作为 Decoder。Encoder 负责将输入序列压缩成指定长度的向量，这个向量就可以看成是这个序列的语义，这个过程称为编码

In [4]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

Using TensorFlow backend.
/Users/zhangjianfeng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
#Batch size 的大小
batch_size = 32  
# 迭代次数epochs
# epochs = 100
epochs = 10
# 编码空间的维度Latent dimensionality 
latent_dim = 256  
# 要训练的样本数
# num_samples = 5000 
num_samples = 500
#设置语料的路径
data_path = '../data/13/files.txt'

In [6]:
# 把语料向量化：
#把数据向量话
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    #print(line)
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = target_text[0:100]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Number of samples: 500
Number of unique input tokens: 716
Number of unique output tokens: 1038
Max sequence length for inputs: 40
Max sequence length for outputs: 36


In [7]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 输出 `encoder_outputs` 
encoder_states = [state_h, state_c]

# 状态 `encoder_states` 
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                       initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 定义模型
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 训练
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# 保存模型
model.save('s2s.h5')

Train on 400 samples, validate on 100 samples
Epoch 1/10
400/400 [==============================] - 5s 14ms/step - loss: 3.1284 - val_loss: 2.7257
Epoch 2/10
400/400 [==============================] - 5s 12ms/step - loss: 2.9159 - val_loss: 2.7183
Epoch 3/10
400/400 [==============================] - 5s 13ms/step - loss: 2.8912 - val_loss: 2.7224
Epoch 4/10
400/400 [==============================] - 5s 13ms/step - loss: 2.8760 - val_loss: 2.7331
Epoch 5/10
400/400 [==============================] - 6s 14ms/step - loss: 2.8687 - val_loss: 2.7478
Epoch 6/10
400/400 [==============================] - 5s 13ms/step - loss: 2.8478 - val_loss: 2.7435
Epoch 7/10
400/400 [==============================] - 5s 13ms/step - loss: 2.8319 - val_loss: 2.7525
Epoch 8/10
400/400 [==============================] - 5s 13ms/step - loss: 2.8315 - val_loss: 2.7671
Epoch 9/10
400/400 [==============================] - 5s 13ms/step - loss: 2.8084 - val_loss: 2.7746
Epoch 10/10
400/400 [========================

/Users/zhangjianfeng/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [8]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [9]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [10]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    #print(states_value)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        # 更新状态
        states_value = [h, c]
    return decoded_sentence

In [11]:
def predict_ans(question):
        inseq = np.zeros((len(question), max_encoder_seq_length, num_encoder_tokens),dtype='float16')
        decoded_sentence = decode_sequence(inseq)
        return decoded_sentence

In [12]:
print('Decoded sentence:', predict_ans("挖机履带掉了怎么装上去"))

Decoded sentence: 油油油油油油油油油油油油油油油

